In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy import stats

In [2]:
data = pd.read_pickle('df_tesco_marker_V3.pkl')

In [3]:
data.shape

(61990, 56)

In [4]:
data.columns

Index(['Site Number', 'Site Name', 'Transaction Date', 'Total Sales Value',
       'Total Baskets', 'Sales Baskets With Fuel', 'Baskets With Fuel',
       'Site ID', 'PBL', 'CRUSO_priceADO', 'CRUSO_priceUL95', 'volumeADO',
       'volumeUL95', 'sales_valADO', 'sales_valUL95', 'totalSalesEsso',
       'pct_dev', 'diff_shop_esso', 'SiteCode', 'SiteName', 'Date',
       'UL 95_ Sales Transaction Volume', 'UL 95_ Approved Price',
       'UL 95_ Marker Price', 'UL 95_ Approved Price Vs Marker Price',
       'ADO_ Sales Transaction Volume', 'ADO_ Approved Price',
       'ADO_ Marker Price', 'ADO_ Approved Price Vs Marker Price',
       'avgUL95VolSite', 'avgADOVolSite', 'diffAdoVol', 'diffUL95Vol',
       'diffAdoVolpct', 'diffUL95Volpct', 'diffAdoPricePct',
       'diffUL95PricePct', 'avgShopSalesFromFuelSiteWise',
       'pctChgShopSalesFromFuel', 'shopSales', 'avgShopSalesSiteWise',
       'ShopSalesPctChgFromAvg', 'AdoApprPriceChgPct', 'UL95AppPriceChgPct',
       'avgFuelSalesEssoSiteWi

In [5]:
# calcualting exclusive shop sales and shop slaes from fuel for sites:
data['exclShopSales'] = data['Total Sales Value']-data['Sales Baskets With Fuel']
data['shopSalesFromFuel'] = data['Sales Baskets With Fuel']-data['totalSalesEsso']

In [6]:
# selecting only required data for elasticity calculation:
seldf = data.loc[:,['Site Number', 'exclShopSales', 'ADO_ Approved Price','ADO_ Marker Price']]

In [7]:
seldf.drop(list(seldf[seldf['exclShopSales'] == 0].index), axis=0, inplace=True)

In [8]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [9]:
def get_plot(df):
    slope, intercept, r_value, p_value, std_err = stats.linregress(df['Actual log(P0/Pbar)'], y=df['Actual log(S0)'])
    line = slope*X+intercept

    df['Fitted log(S0)'] = np.concatenate(line)

    fig = go.Figure(data=[
        go.Scatter(x= df['Actual log(P0/Pbar)'], y=df['Actual log(S0)'],mode='markers', name='Actual', marker_color = 'gray'),
        go.Line(x= df['Actual log(P0/Pbar)'], y=df['Fitted log(S0)'], name='Fit', marker_color = 'firebrick')    
    ])

    fig.update_layout(
        #title = selectedCountry+' - Road Fuel Demand',
        yaxis_title="log(S0)",
        xaxis_title="log(P0/Pbar)",
        plot_bgcolor='rgba(0,0,0,0)'
    )

    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    #fig.show()
    #fig.write_image("fig1.pdf")
    return fig

In [10]:
elasticity = []
logSbar    = []
corrCoef   = []
pVal       = []
rmse       = []
r2         = []
mape       = []


for i in range(len(seldf['Site Number'].unique())): #for all sites 
    df = seldf[seldf['Site Number'] == seldf['Site Number'].unique()[i]]
    df.drop('Site Number', axis=1, inplace=True)
    df.columns = ['shopSales', 'P0', 'Pbar']
    data_df = pd.DataFrame()
    y = np.log(df['shopSales'].dropna().values).reshape(-1,1)
    pdiv = df['P0']/df['Pbar']
    X = np.log(pdiv.dropna()).values.reshape(-1,1)
    data_df = pd.concat([data_df, pd.DataFrame({'Actual log(S0)': y.flatten(), 'Actual log(P0/Pbar)': X.flatten()})], axis=0)
    #get_plot(data_df)
    slope, intercept, r_value, p_value, std_err = stats.linregress(data_df['Actual log(P0/Pbar)'], y=data_df['Actual log(S0)'])
    elasticity.append(slope)
    logSbar.append(intercept)
    corrCoef.append(r_value)
    pVal.append(p_value)
    rmse.append(round(np.sqrt(metrics.mean_squared_error(y, slope*X+intercept)),4))
    r2.append(round(metrics.r2_score(y, slope*X+intercept),4))
    mape.append(round(mean_absolute_percentage_error(y,slope*X+intercept),4))   

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/app-root/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning:



In [11]:
# df = data_df
# slope, intercept, r_value, p_value, std_err = stats.linregress(df['Actual log(P0/Pbar)'], y=df['Actual log(S0)'])
# line = slope*X+intercept

# df['Fitted log(S0)'] = np.concatenate(line)

# fig = go.Figure(data=[
#     go.Scatter(x= df['Actual log(P0/Pbar)'], y=df['Actual log(S0)'],mode='markers', name='Actual', marker_color = 'gray'),
#     go.Line(x= df['Actual log(P0/Pbar)'], y=df['Fitted log(S0)'], name='Fit', marker_color = 'firebrick')    
# ])

# fig.update_layout(
#     #title = selectedCountry+' - Road Fuel Demand',
#     yaxis_title="log(S0)",
#     xaxis_title="log(P0/Pbar)",
#     plot_bgcolor='rgba(0,0,0,0)'
# )

# fig.update_xaxes(showgrid=False)
# fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
# fig.show()

In [12]:
# slope, intercept, r_value, p_value, std_err = stats.linregress(data_df['Actual log(P0/Pbar)'], y=data_df['Actual log(S0)'])
# elasticity.append(slope)
# logSbar.append(intercept)
# corrCoef.append(r_value)
# pVal.append(p_value)
# rmse.append(round(np.sqrt(metrics.mean_squared_error(y, slope*X+intercept)),4))
# r2.append(round(metrics.r2_score(y, slope*X+intercept),4))
# mape.append(round(mean_absolute_percentage_error(y,slope*X+intercept),4))   

In [13]:
result_df = pd.DataFrame({'elasticity': elasticity, 'constant': logSbar, 'p-value':pVal, 'r2':r2, 'mape': mape, 'rmse':rmse})

In [14]:
result_df

elasticity  constant   p-value      r2    mape    rmse
0     -1.446985  9.469448  0.261844  0.0038  1.9732  0.2192
1      0.242203  9.418377  0.746566  0.0003  0.7998  0.0934
2      1.463604  8.635636  0.173667  0.0058  1.0521  0.1148
3      1.040821  9.348888  0.072709  0.0101  0.8510  0.1036
4     -3.245308  9.078700  0.001054  0.0348  1.1583  0.1430
..          ...       ...       ...     ...     ...     ...
191   -0.166842  9.061605  0.870366  0.0001  1.0596  0.1272
192   -2.226057  9.385415  0.043279  0.0127  0.8558  0.1053
193   -0.069791  8.758953  0.956726  0.0000  1.0844  0.1233
194   -0.043380  8.971376  0.945438  0.0000  0.8059  0.1040
195    0.643100  9.569199  0.650207  0.0007  2.1195  0.2246

[196 rows x 6 columns]

In [15]:
result_df[result_df['p-value'] < 0.05].shape

(65, 6)

In [16]:
import plotly.express as px
fig = px.histogram(result_df, x="elasticity", nbins=20)
fig.show()

In [17]:
result_df.to_csv("ADO_excShopSales_elasticity")